In [ ]:
from dotenv import load_dotenv
import os
import pandas as pd
import requests

def convert(decimal):
    if decimal >= 2.0:
        return '+' + str(int((decimal - 1) * 100))
    else:
        return int(-100 / (decimal - 1))

BETS = pd.DataFrame(columns=['sport','team','odds'])
load_dotenv()
API_KEY = os.getenv('ODDS_API_KEY')

ACTIVE_SPORTS = []
SPORTS = ['baseball_mlb','basketball_nba','basketball_ncaab','americanfootball_ncaab','americanfootball_nfl','icehockey_nhl']
request = requests.get(f"https://api.the-odds-api.com/v4/sports/?apiKey={API_KEY}")
data = request.json()
for sport in data:
    if sport['key'] in SPORTS and sport['active']:
        ACTIVE_SPORTS.append(sport['key'])

for sport in ACTIVE_SPORTS:
    request = requests.get(f"https://api.the-odds-api.com/v4/sports/{sport}/odds/?apiKey={API_KEY}&regions=us,us2&markets=h2h")
    data = request.json()

    for game in data:
        home_team = game['home_team']
        away_team = game['away_team']
        home_odds = []
        away_odds = []
        fliff_h_odds = 0
        fliff_a_odds = 0
        print(f"GAME: {home_team} vs {away_team}")
        for book in game['bookmakers']:
            if book['markets'][0]['outcomes'][0]['name'] == home_team:
                home_odd = book['markets'][0]['outcomes'][0]['price']
                away_odd = book['markets'][0]['outcomes'][1]['price']
            else:
                home_odd = book['markets'][0]['outcomes'][1]['price']
                away_odd = book['markets'][0]['outcomes'][0]['price']
            if book['title'] == 'Fliff':
                fliff_h_odds = home_odd
                fliff_a_odds = away_odd
            home_odds.append(home_odd)
            away_odds.append(away_odd)
        h_avg = sum(home_odds) / len(home_odds)
        a_avg = sum(away_odds) / len(away_odds)
        print(f"GAME H AVG: {h_avg} ||| FLIFF {fliff_h_odds}")
        print(f"GAME A AVG: {a_avg} ||| FLIFF {fliff_a_odds}")
        if fliff_h_odds != 0 and fliff_h_odds >= h_avg:
            print(f"BET FOUND: {home_team} at {fliff_h_odds}")
            odds = convert(fliff_h_odds)
            BETS.loc[len(BETS)] = {'sport': sport, 'team': home_team, 'odds': odds}
        if fliff_a_odds != 0 and fliff_a_odds >= a_avg:
            print(f"BET FOUND: {away_team} at {fliff_a_odds}")
            odds = convert(fliff_a_odds)
            BETS.loc[len(BETS)] = {'sport': sport, 'team': away_team, 'odds': odds}
        print()
BETS